# Conectar al vostre "Google Drive"
Per sintetitzar les frases és a dir fer un "inference" necessitem els models de Tacotron2 i també del vocoder. La manera de fer-ho és a partir dels enllaços afegir els fitxers al vostre google drive.

Un cop els fitxers estan a les nostres carpetes, podem donar els permisos a aquest "notebook" per accedir a les carpetes del drive. D'aquesta manera els fitxers seran visibles pel codi.

Els enllaços són:
* [Model de Catotron](https://drive.google.com/open?id=1-fdWV-aH5nIRv1rZKQYInsRes2At74xG
), entrenat amb [Festcat](http://festcat.talp.cat/download.php) veu, Ona
* [Model de Waveglow](https://drive.google.com/open?id=1WsibBTsuRg_SF2Z6L6NFRTT-NjEy1oTx), entrenat amb LJ Speech
* [Model de MelGAN](https://drive.google.com/file/d/1U3LeuaMIVoRvMvfwlHjsRJPWhgTzeIBh/view), entrenat amb dades de Festcat
* [Dades de Festcat](https://drive.google.com/open?id=1Mu5qxJovWzMRzXwHLdS2s69rDXhUzzB9
); només la veu Pau, procecessat per Tacotron2

In [0]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p "/content/drive/My Drive/tacotron_models"
!ls "/content/drive/My Drive/tacotron_models"

# Importar el codi
El "notebook" de colab ens deixa executar ordres del terminal d'un linux, mitjançant el `!` i `%`. A més, els servidors del colab venen amb certes aplicacions instal·lades com a CUDA i git.

Per importar el codi, farem un clon de github.

In [0]:
import sys
!git clone https://github.com/collectivat-dev/tacotron2 catotron
%cd catotron
!git submodule init; git submodule update
sys.path.append('/content/catotron/waveglow')

# Instal·lació de les llibreries i cridar-les

In [0]:
%%bash
pip install numpy scipy librosa unidecode inflect librosa tensorboardX

In [4]:
# Generic libraries
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd
from scipy.io.wavfile import write

import numpy as np
import torch
from audio_processing import griffin_lim

# tacotron2 modules
from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT

import distributed
from train import load_model
from text import text_to_sequence
#from denoiser import Denoiser

# Carregar els models
Per generar una veu, Tacotron2 necessita dos passos: el primer generar els mel espectrogrames i el segon generar les ones a partir dels espectrogrames. Per aquesta raó necessitem dos models un per Tacotron2 un altre pel Vocoder. En aquest cas un model de Waveglow.

Amb aquest pas estem carregant els dos models a la memòria.

In [0]:
hparams = create_hparams()
rate = 22050
hparams.sampling_rate = rate

checkpoint_path = "/content/drive/My Drive/tacotron_models/upc_ona_tacotron2.pt"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval().half()

In [0]:
waveglow_path = '/content/drive/My Drive/tacotron_models/waveglow_256channels_ljs_v2.pt'
waveglow = torch.load(waveglow_path)['model']
waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
#denoiser = Denoiser(waveglow)

# fix for the "AttributeError: 'ConvTranspose1d' object has no attribute 'padding_mode'"
for m in waveglow.modules():
    if 'Conv' in str(type(m)):
        setattr(m, 'padding_mode', 'zeros')

# Sintetitzar la veu

Aquí introduirem un text, per generar la veu.

Fixeu-vos la crida als dos models. Podem escoltar el resultat dins del "notebook" mitjançant el modul `ipython display`.

In [0]:
# introduce the text
text = "En temps difícils, hem de veure els nostres èxits i augmentar el nostre coratge."

# preprocessing
sequence = np.array(text_to_sequence(text, ['catalan_cleaners']))[None, :]
sequence = torch.from_numpy(sequence).to(device='cuda', dtype=torch.int64)

# run the models
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet)
audio_numpy = audio[0].data.cpu().numpy()

# make audio listenable
ipd.Audio(audio_numpy, rate=rate)


In [0]:
!mkdir -p "/content/drive/My Drive/test_wavs/"
audio_numpy /= np.max(np.abs(audio_numpy))
write('/content/drive/My Drive/test_wavs/catotron_nvidia.wav', rate, audio_numpy.astype(np.float32))

# Fer servir l'algorisme Griffin-Lim

L'algorisme de Griffin-Lim facilita sintetitzar la veu sense la necessitat d'un vocoder entrenat per xarxes neuronals. Quan estem experimentant amb dades noves, i no tenim cap vocoder entrenat, aquest algorisme ajuda fer un control de qualitat ràpid. 

In [0]:
def infer(checkpoint_path, griffin_iters, text):
    hparams = create_hparams()
    hparams.sampling_rate = 22050

    model = load_model(hparams)
    model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
    _ = model.cuda().eval()#.half()

    sequence = np.array(text_to_sequence(text, ['catalan_cleaners']))[None, :]
    sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()

    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)

    taco_stft = TacotronSTFT(hparams.filter_length, hparams.hop_length, hparams.win_length, sampling_rate=hparams.sampling_rate)

    mel_decompress = taco_stft.spectral_de_normalize(mel_outputs_postnet)
    mel_decompress = mel_decompress.transpose(1, 2).data.cpu()
    spec_from_mel_scaling = 1000
    spec_from_mel = torch.mm(mel_decompress[0], taco_stft.mel_basis)
    spec_from_mel = spec_from_mel.transpose(0, 1).unsqueeze(0)
    spec_from_mel = spec_from_mel * spec_from_mel_scaling

    audio = griffin_lim(torch.autograd.Variable(spec_from_mel[:, :, :-1]), taco_stft.stft_fn, griffin_iters)

    audio = audio.squeeze()
    audio = audio.cpu().numpy()
    return audio


In [0]:
audio_griffin = infer(checkpoint_path, 60, text)
ipd.Audio(audio_griffin, rate=rate)

In [0]:
!mkdir -p "/content/drive/My Drive/test_wavs/"
audio_griffin /= np.max(np.abs(audio_griffin))
write('/content/drive/My Drive/test_wavs/catotron_griffin.wav', hparams.sampling_rate, audio_griffin)

# Generar gràfiques

Per donar una ullada a l'alineament i els espectrograma, fem servir matplotlib.

In [0]:
def plot_data(data, figsize=(16, 16)):
    fig, axes = plt.subplots(len(data), 1, figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom', 
                       interpolation='none')
    plt.savefig('/content/result.png')

In [0]:
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))